In [ ]:
import dash
import dash_daq as daq
from dash import dcc
from dash import html
import dash_table_experiments as dt
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import time
import pandas as pd
from pycoingecko import CoinGeckoAPI
import plotly.express as px
import pymongo
from pymongo import MongoClient
from dash.dependencies import Input, Output, State
from datetime import datetime
import random

In [ ]:
app = dash.Dash(external_stylesheets=[dbc.themes.SLATE])

cg = CoinGeckoAPI()

client = MongoClient("mongodb://127.0.0.1:27017/")
mydb = client["dashboard"]

collection = mydb.gecko

In [ ]:
#OHLC------------------------------------------------------------------------------------------------------------------------
def ohlc_data_by_id(coin, currency, days):
    data = cg.get_coin_ohlc_by_id(id=coin, vs_currency=currency, days=days)
    df = pd.DataFrame(data)
    df.columns = ["timestamp", "open", "high", "low", "close"]
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    # print(df)
    fig = go.Figure(data=go.Ohlc(x=df["timestamp"],
                    open=df["open"],
                    high=df["high"],
                    low=df["low"],
                    close=df["close"]))           
    # fig.add_trace(go.Scatter("
    fig.update_layout(barmode = "stack", 
                      paper_bgcolor="#272b30", 
                      plot_bgcolor="#272b30",
                      xaxis=dict(rangeselector=dict(buttons=list([
                          dict(count=1,
                               label='1h',
                               step='hour',
                               stepmode='backward'),                          
                          dict(count=1,
                               label='1d',
                               step='day',
                               stepmode='backward'),
                          dict(count=6,
                               label='1m',
                               step='month',
                               stepmode='backward'),
                          # dict(count=1,
                          #      label='YTD',
                          #      step='year',
                          #      stepmode='todate'),
                          dict(count=1,
                               label='1y',
                               step='year',
                               stepmode='backward'),
                          dict(step='all')
                      ])
        ),rangeslider=dict(), type="date"))
                      
    fig.update_xaxes(color = "#aaa", showgrid = False, visible = True, title_text="Shiba-Inu/USD", title_font=dict(size=20, color="#aaa"))
    fig.update_yaxes(color = "#aaa", tickformat=".7f")
    
    return fig

#Sentimen Analisys-----------------------------------------------------------------------------------------------------------------------
def sentiment_daily_data_by(coin):   
    coin_data = cg.get_coin_by_id(
        coin, 
        market_data='true', 
        sparkline='false',
        tickers='false',
        developer_data='false',
        localization='false',    
    )
     
    data = {
        "id": [coin_data["id"]],
        "name": [coin_data["name"]],
        "last_updated": [coin_data["last_updated"]],
        "positive": [coin_data["sentiment_votes_up_percentage"]],
        "negative": [coin_data["sentiment_votes_down_percentage"]]
    }
    
    df = pd.DataFrame(data)    
    
    return df

def get_sentiment_daily_data():
    sentiment_shiba_inu = sentiment_daily_data_by("shiba-inu")
    sentiment_bitcoin = sentiment_daily_data_by("bitcoin")
    sentiment_ethereum =sentiment_daily_data_by("ethereum")

    fig = go.Figure()
    fig.add_trace(go.Bar(
        x = [sentiment_shiba_inu["positive"][0], sentiment_bitcoin["positive"][0], sentiment_ethereum["positive"][0]],    
        y = ["Shiba Inu", "Bitcoin", "Ethereum"],
        name = "Positive",
        orientation = "h",
        texttemplate= [sentiment_shiba_inu["positive"][0],sentiment_bitcoin["positive"][0], sentiment_ethereum["positive"][0]],
        textposition="inside",
        textfont_color="#aaa",
        marker = dict(color="green")
    ))

    fig.add_trace(go.Bar(
        x = [sentiment_shiba_inu["negative"][0], sentiment_bitcoin["negative"][0], sentiment_ethereum["negative"][0]],
        y = ["Shiba Inu", "Bitcoin", "Ethereum"],
        name = "Negative",
        orientation = "h", 
        texttemplate= [sentiment_shiba_inu["negative"][0], sentiment_bitcoin["negative"][0], sentiment_ethereum["negative"][0]],
        textposition="inside",
        textfont_color="#aaa",
        marker = dict(color="red")
    ))

    # fig.update_layout(barmode = "stack", height = 400, width = 800, template = "simple_white", paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
    fig.update_layout(barmode = "stack", 
                      paper_bgcolor="#272b30", 
                      plot_bgcolor="#272b30", 
                      font_color="#aaa",
                      title={'text': "Sentiment Analisys", 'y':0.115, 'x':0.5, 'xanchor': 'center', 'yanchor': 'bottom'},
                      title_font=dict(size=20, color="#aaa"))
    fig.update_xaxes(showgrid = False, visible = False, color="#aaa")
    fig.update_yaxes(showgrid = False, visible = True)
    
    return fig

#---------------------------------------------------------------------------------------------------------------------------------------

#Price/Sentiment
def get_coin_values(coin, currency, days):
    data = cg.get_coin_market_chart_by_id(id=coin, vs_currency=currency, days=days)
    df = pd.DataFrame(data)
    
    timestamp = df["prices"].apply(pd.Series)[0]    
    # print(timestamp)
    prices = df["prices"].apply(pd.Series)[1]
    market_caps = df["market_caps"].apply(pd.Series)[0]
    total_volumes = df["total_volumes"].apply(pd.Series)[0]    
    final = pd.concat([timestamp, prices, market_caps, total_volumes], axis=1, keys=["timestamp", "prices", "market_caps", "total_volumes"])
    final["timestamp"] = pd.to_datetime(final["timestamp"], unit="ms")
    final.insert(0, column="id", value=coin)
    
    fig = px.line(final, x=final["timestamp"], y=["prices"])
    fig.update_layout(paper_bgcolor="#272b30", 
                      plot_bgcolor="#272b30",
                      xaxis=dict(rangeselector=dict(buttons=list([
                          dict(count=1,
                               label='1h',
                               step='hour',
                               stepmode='backward'),                          
                          dict(count=1,
                               label='1d',
                               step='day',
                               stepmode='backward'),
                          dict(count=6,
                               label='1m',
                               step='month',
                               stepmode='backward'),
                          # dict(count=1,
                          #      label='YTD',
                          #      step='year',
                          #      stepmode='todate'),
                          dict(count=1,
                               label='1y',
                               step='year',
                               stepmode='backward'),
                      ])
        ),rangeslider=dict(), type="date"))
    
    fig.update_xaxes(color="#aaa", showgrid=False, visible=True, title_text="Price/Sentiment", title_font=dict(size=20, color="#aaa"))
    fig.update_yaxes(color="#aaa", tickformat=".7f", title_text="")
    
    return fig


In [ ]:
app.layout = \
dbc.Container\
([
    dbc.Row([
        html.H2("Crypto Analisys Sentiment-Based")
    ]),
    dbc.Row([
        dbc.Col([html.Br(),html.Br(),html.Br(),html.Br(),html.Br(),
             daq.Gauge(id='gauge',
                       color={"gradient":True,"ranges":{"red":[0,60],"yellow":[60,80],"green":[80,100]}},                       
                       value=50,
                       label="SELL-HODL-BUY",
                       labelPosition="bottom",  
                       size=300,
                       min=0,
                       max=100,
                       scale={"custom":{"20": "SELL", "50": "HODL", "80":"BUY"}}
                      ),
            dcc.Interval(id='g-trigger', interval=20000)],
            lg=4, md=12, xs=12),
            # width=4),
        dbc.Col([
            dcc.Graph(id="grafica_2"), 
            dcc.Interval(id="trigger", interval=20000)],
            lg=8, md=12, xs=12),
            # width=8)
    ]),       
    dbc.Row([
        dbc.Col([
            dcc.Graph(figure=get_sentiment_daily_data())],
            lg=4, md=12, xs=12),
            # width=4),
        dbc.Col([
            dcc.Graph(figure=get_coin_values("shiba-inu", "usd", 90)),            
            ],            
            lg=8, md=12, xs=12),
            # # width=8),
    ]),
    dbc.Row([
        dbc.Col([
            html.H6("Our recomendations are based on sentimental analisis from tweeter and reddit"),
            html.H6("Our data source are tweeter, reddit and coingecko API")            
        ], width=12)
    ])    
])

@app.callback(Output('gauge', 'value'),
              Input('g-trigger', 'n_intervals'))
def update_gauge(value):
    return random.randint(0,100)

@app.callback(Output('grafica_2', 'figure'),
              Input('trigger', 'n_intervals'))

def price_graph(n):
    coin_data = cg.get_price("bitcoin", "usd")
    
    register = {
        # "timestamp": datetime.today().strftime("%d/%m/%Y %H:%M:%S"),
        "timestamp": datetime.today().strftime("%Y/%m/%d %H:%M:%S"),
        # "timestamp": time.time_ns(),
        "id": list(coin_data)[0],
        "price": coin_data["bitcoin"]["usd"]
    }
        
    collection.insert_one(register)
    
    df= pd.DataFrame(list(mydb.gecko.find()))    
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    
    fig = px.line(df, x=df["timestamp"], y="price")    
    fig.update_xaxes(tickformat="%H:%M", nticks=10, color="#aaa", showgrid=False, visible=True, title_text="BTC/USD", title_font=dict(size=20, color="#aaa"))
    fig.update_yaxes(color="#aaa", tickformat=".f", title_text="")
    fig.update_layout(paper_bgcolor="#272b30", 
                      plot_bgcolor="#272b30",  
                      xaxis=dict(rangeselector=dict(buttons=list([
                          dict(count=1,
                               label='1h',
                               step='hour',
                               stepmode='backward'),                          
                          dict(count=1,
                               label='1d',
                               step='day',
                               stepmode='backward'),
                          dict(count=6,
                               label='1m',
                               step='month',
                               stepmode='backward'),
                          dict(count=1,
                               label='1y',
                               step='year',
                               stepmode='backward'),
                        ])                                                                                    
                    ),rangeslider=dict(), type="date"
                    ))                      
                         
    return fig



if __name__ == "__main__":
    app.run_server(debug=False, port=8052, host='172.16.51.7', use_reloader=False)